필요한 라이브러리 설치와 임포트

In [2]:
!pip install sentencepiece
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install tqdm pandas
!pip install torch
!pip install sentencepiece
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 15.3 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292695 sha256=f11f236d1c9a9f0750a516dca9f998bead13d054f65fa2314dd4f11d0dc9a539
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp


In [3]:
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-z07i5c4t/kobert-tokenizer_7f28a704ff3941e895554ebcc73557cb
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-z07i5c4t/kobert-tokenizer_7f28a704ff3941e895554ebcc73557cb
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=ef54d2aae056570aa03c610652c03335cd1326e0e32295ad3110855942b9a618
  Stored in directory: /tmp/pip-ephem-wheel-cache-4i91haq5/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [4]:
from kobert_tokenizer import KoBERTTokenizer
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader

In [6]:
import gluonnlp as nlp
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import glob
import os

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
device_type = 'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device_type)

각 폴더에서 텍스트 저장한 엑셀파일 위치 가져오기

In [9]:
section_folders = glob.glob('/content/drive/MyDrive/NaverNews/*')
section_folders

['/content/drive/MyDrive/NaverNews/economy',
 '/content/drive/MyDrive/NaverNews/world',
 '/content/drive/MyDrive/NaverNews/society',
 '/content/drive/MyDrive/NaverNews/science',
 '/content/drive/MyDrive/NaverNews/politics',
 '/content/drive/MyDrive/NaverNews/life']

In [10]:
root = '/content/drive/MyDrive/NaverNews'
sections = os.listdir(root)
sections

['economy', 'world', 'society', 'science', 'politics', 'life']

In [11]:
text_paths = []
for folder, section in zip(section_folders, sections):
  text_path = folder + '/' + section + 'text' + '.xlsx'
  text_paths.append(text_path)

text_paths

['/content/drive/MyDrive/NaverNews/economy/economytext.xlsx',
 '/content/drive/MyDrive/NaverNews/world/worldtext.xlsx',
 '/content/drive/MyDrive/NaverNews/society/societytext.xlsx',
 '/content/drive/MyDrive/NaverNews/science/sciencetext.xlsx',
 '/content/drive/MyDrive/NaverNews/politics/politicstext.xlsx',
 '/content/drive/MyDrive/NaverNews/life/lifetext.xlsx']

각 섹션별 인덱스로 분석에 사용할 텍스트 가져오기, label - y 페어 만들기

In [12]:
idx_dictionary = {
    'politics' : range(200),
    'society' : range(200),
    'science' : range(200),
    'life' : range(200),
    'world' : range(200),
    'economy' : range(200)
}

In [ ]:
# label_to_y = {section : idx for idx, section in enumerate(sections)}
# label_to_y

In [13]:
label_to_y = {
    'politics': 0,
    'society': 1,
    'science': 2,
    'life': 3,
    'world': 4,
    'economy': 5
}
label_to_y

{'politics': 0,
 'society': 1,
 'science': 2,
 'life': 3,
 'world': 4,
 'economy': 5}

전체 데이터 : 제목 + 본문 텍스트 (X), 라벨과 일대일대응된 숫자(y)

In [14]:
overall_data = []

for section_folder in section_folders:
  section = os.path.basename(section_folder)
  section_textpath = section_folder + '/' + section + 'text' + '.xlsx'

  section_texts_raw = pd.read_excel(section_textpath)


  idx = idx_dictionary[section]

  # X
  section_texts = section_texts_raw.iloc[idx]
  articles = section_texts['title'] + section_texts['body']

  # y
  y = label_to_y[section]

  for article in articles:
    data = []
    data.append(article)
    data.append(y)

    overall_data.append(data)

In [15]:
overall_data[1]

["국민연금, 카카오 투자목적 일반투자로 상향…적극적 주주활동 예고(서울=뉴스1) 김정현 기자 = 국민연금이 카카오(35720) 투자 목적을 단순투자에서 일반투자로 상향했다.1일 금융감독원 전자공시시스템에 따르면 국민연금공단은 카카오 주식의 보유목적을 단순투자목적에서 일반투자목적으로 변경하고, 주식 지분율도 6.36%에서 5.42%로 줄인다고 공시했다.국민연금공단은 이날 카카오 계열사인 카카오페이(377300)의 보유목적도 일반투자로 변경하고, 보유 비율도 5.02%에서 4.45%로 축소했다.기관 투자자의 기업 지분 보유 목적은 △단순투자 △일반투자 △경영 참여 세단계로다. 일반투자는 정관 변경, 위법행위에 대한 임원진 해임 청구 등 단순투자보다 적극적인 주주 활동을 할 수 있다.국민연금공단의 이같은 행보는 최근 카카오 그룹이 전방위로 수사·금융당국의 압박을 받고 있는 상황과 관련해 주주활동을 제고하려는 움직임으로 해석된다. 국민연금은 현재 카카오의 4대 주주다.본시장특별사법경찰(특사경)은 최근 주가조작 혐의로 배재현 카카오 투자총괄대표 등 관계자들을 자본시장법 위반 혐의로 검찰에 송치했다. 김범수 카카오 미래이니셔티브 센터장에 대한 조사도 계속되고 있다.다른 카카오 계열사인 카카오모빌리티 역시 '매출 부풀리기' 의혹과 관련해 분식회계에 대한 감리가 진행 중이다.",
 5]

train / test split

In [16]:
# train / test set split -> 나중에 stratified K-fold cv 라이브러리로 대체
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(overall_data, test_size = 0.2, shuffle = True)

In [17]:
train_data[1]

['박민 KBS 사장 후보자 인사청문회 7일 실시    (서울=연합뉴스) 최평천 기자 = 박민 KBS 사장 후보자에 대한 국회 인사청문회가 오는 7일 열린다.    국회 과학기술정보방송통신위원회는 1일 전체회의를 열어 이러한 내용의 인사청문 계획서를 채택했다.    여야는 청문회에 참석할 증인 명단은 아직 합의하지 못했다. 야당은 서기석 KBS 이사장 등을 증인으로 채택할 것을 요구하고 있다.    윤석열 대통령은 지난달 17일 "KBS가 국민의 방송으로 거듭나기 위해 필요한 개혁을 과감히 추진해 나갈 것으로 기대된다"며 박 후보자 인사청문요청안을 국회로 보냈다.    국민의힘은 박 후보자에 대해 "1991년 문화일보 기자로 30년 넘게 언론 현장에서 다양한 경험을 쌓았고, 다변화된 언론 환경에 맞춰 KBS를 혁신해 갈 수 있기를 기대한다"고 밝힌 바 있다.    더불어민주당은 KBS 사장 후보 선정 과정의 절차적 문제점과 자질 부족을 이유로 박 후보자 임명을 반대하고 있다.    민주당은 지난 4일 열린 KBS 이사회에서 사장 후보 3명 중 박 후보자가 과반 표를 얻지 못하자 서 이사장이 결선 투표 진행을 미뤘고, 이후 임시 이사회에서 박 후보자를 최종 후보자로 결정한 것을 문제 삼고 있다.pc@yna.co.kr',
 1]

KoBERT 불러오기

In [18]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)

        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')

In [19]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [20]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')

tokenizer_config.json:   0%|          | 0.00/432 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/371k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/244 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [21]:
# kobert 공식 git에 있는 get_kobert_model 선언
def get_kobert_model(model_path, vocab_file, ctx="cpu"):
    bertmodel = BertModel.from_pretrained(model_path)
    device = torch.device(ctx)
    bertmodel.to(device)
    bertmodel.eval()
    vocab_b_obj = nlp.vocab.BERTVocab.from_sentencepiece(vocab_file,
                                                         padding_token='[PAD]')
    return bertmodel, vocab_b_obj


In [22]:
from transformers import BertModel
bertmodel, vocab = get_kobert_model('skt/kobert-base-v1', tokenizer.vocab_file)
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower = False)

config.json:   0%|          | 0.00/535 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/369M [00:00<?, ?B/s]

In [23]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes = 6,   # 6 classes
                 dr_rate = None,
                 params = None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p = dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device),return_dict = False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

데이터를 파이토치에 맞는 데이터셋으로 변환

In [24]:
max_len = 200
batch_size = 8

In [25]:
train_dataset = BERTDataset(dataset = train_data, sent_idx = 0, label_idx = 1, bert_tokenizer = tokenizer, vocab = vocab, max_len = max_len, pad = True, pair = False)
test_dataset = BERTDataset(dataset = test_data, sent_idx = 0, label_idx = 1, bert_tokenizer = tokenizer, vocab = vocab, max_len = max_len, pad = True, pair = False)

데이터 로딩

In [26]:
train_dataloader = DataLoader(train_dataset, batch_size = batch_size)
test_dataloader = DataLoader(test_dataset, batch_size = batch_size)

모델 정의, 손실함수와 강하법 선택

In [27]:
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [28]:
model = BERTClassifier(bertmodel,  dr_rate = 0.5).to(device)

In [29]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr = learning_rate)
loss_fn = nn.CrossEntropyLoss() # 다중분류를 위한 loss function

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps = warmup_step, num_training_steps = t_total)


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [30]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [31]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [32]:
train_history = []
test_history = []
loss_history = []

for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)

        # print(label.shape, out.shape)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
            train_history.append(train_acc / (batch_id+1))
            loss_history.append(loss.data.cpu().numpy())
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    # train_history.append(train_acc / (batch_id+1))

    # .eval() : nn.Module에서 train time과 eval time에서 수행하는 다른 작업을 수행할 수 있도록 switching 하는 함수
    # 즉, model이 Dropout이나 BatNorm2d를 사용하는 경우, train 시에는 사용하지만 evaluation을 할 때에는 사용하지 않도록 설정해주는 함수
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length = valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
    test_history.append(test_acc / (batch_id+1))

  0%|          | 0/120 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 1.8223624229431152 train acc 0.125
epoch 1 train acc 0.3177083333333333


  0%|          | 0/30 [00:00<?, ?it/s]

epoch 1 test acc 0.6291666666666667


  0%|          | 0/120 [00:00<?, ?it/s]

...

epoch 9 test acc 0.6833333333333333


  0%|          | 0/120 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.023642076179385185 train acc 1.0
epoch 10 train acc 0.9229166666666667


  0%|          | 0/30 [00:00<?, ?it/s]

epoch 10 test acc 0.6791666666666667


In [33]:
torch.save(model.state_dict(),'/content/drive/My Drive/KoBERT.pth')